In [1]:
import pandas as pd

In [2]:
df = pd.read_json("data/test_data_json.json")

In [6]:
columns_to_transform = [
            "shapes",
            "property_contacts",
            "tenants",
            "sales",
            "taxes",
            "description",
        ]

## Transform

### functions

In [11]:
def create_description(x: object) -> object:
        """Creates description from summary and details columns.

        Args:
            x (two dataframe series): dataframe row contain summary and details data. Passed using axis=1 in df lambda function

        Returns:
            description (dataframe series): combined description series of summary and details columns
        """
        description = {**x[0], **x[1]}

        return description
    
def insert_value(x: object, value_name: str):
        """Inserts value into embedded json

        Args:
            x (pandas series): two column pandas series containing the value to insert and the embedded json object
            value_name (str): _description_

        Returns:
            current_dict: embedded json with new value inserted
        """
        value = x[0]
        current_dict = x[1]

        current_dict[value_name] = value

        return current_dict

### Transform denormalized table

In [12]:
def transform_data(data: object) -> object:
    """Inputs reonomy_id and msa name into unnormalized data frame, and drops unwanted columns.

    This function expects a dataframe containing all columns with embdedded json objects and will do three things:
        - Drop unwanted columns
        - Add reonomy_id and msa to each embedded json object
        - Combined summary and details into new description column

    Args:
        data (dataframe): Unnormalized data frame, containing all embdedded json columns as dataframe columns

    Returns:
        df (dataframe): Unnormalized data frame with dropped columns, new description column and reonomy_id + msa in each column json object
    """

    df = data.copy()
    df["description"] = df[["summary", "details"]].apply(
        create_description, axis=1
    )
    df["reonomy_id"] = df["scraping_details"].apply(lambda x: x["property_id"])
    df["msa"] = df["details"].apply(lambda x: x["msa_name"])

    columns_to_keep = columns_to_transform + ["reonomy_id", "msa"]
    df = df[columns_to_keep]

    for column in columns_to_transform:
        df[column] = df[["reonomy_id", column]].apply(
            lambda x: insert_value(x, "reonomy_id"), axis=1
        )
        df[column] = df[["msa", column]].apply(
            lambda x: insert_value(x, "msa"), axis=1
        )

    return df


In [13]:
data_transformed = transform_data(df)

### tenants

In [15]:
def handle_tenants(tenants_dict):
        """Transforms the tenants dictionary into a normalized pandas tables

        Args:
            tenants_dict (dict): Dictationary containing tenants data

        Returns:
            tenants_table (dataframe): normalized pandas dataframe containing tenants data
        """

        tenants_tbl = pd.json_normalize(tenants_dict, max_level=0).T
        tenants_tbl = tenants_tbl.rename(columns={0: "tenants"})

        tenants_table = pd.DataFrame()

        row_length = tenants_tbl.shape[0]

        for i in range(row_length):
            row = tenants_tbl.iloc[i]["tenants"]

            tenants_df = pd.json_normalize(
                row, max_level=0, record_path=["tenants"], meta=["reonomy_id", "msa"]
            )

            tenants_table = pd.concat([tenants_table, tenants_df])

        tenants_table = tenants_table.reset_index(drop=True)

        return tenants_table

In [17]:
df_dict = data_transformed.to_dict()

In [20]:
tenants = handle_tenants(df_dict['tenants'])

In [22]:
tenants.columns

Index(['business_name', 'ceo_full_name', 'ceo_title', 'city', 'contacts',
       'current_occupant', 'duns', 'employee_total', 'naics', 'naics2',
       'naics2_description', 'naics_description', 'sales_volume', 'sic',
       'sic2', 'sic2_description', 'sic_description', 'state',
       'street_address', 'telephone_number', 'type_of_location',
       'year_started', 'zip_code', 'zip_code4', 'reonomy_id', 'msa', 'website',
       'tradestyle_name', 'mailing_city', 'mailing_state',
       'mailing_street_address', 'mailing_zip_code', 'mailing_zip_code4'],
      dtype='object')

### Contacts

In [29]:
def handle_contacts(contacts_dict):
        """Transforms the contacts dictionary into a normalized pandas tables

        Args:
            contacts_dict (dict): Dictationary containing contacts data

        Returns:
            contacts_table (dataframe): normalized pandas dataframe containing contacts data
        """

        contacts_tbl = pd.json_normalize(contacts_dict, max_level=0).T
        contacts_tbl = contacts_tbl.rename(columns={0: "property_contacts"})

        contacts_table = pd.DataFrame()

        row_length = contacts_tbl.shape[0]

        for i in range(row_length):
            row = contacts_tbl.iloc[i]

            contacts_df = pd.json_normalize(row, max_level=0)

            contacts_table = pd.concat([contacts_table, contacts_df])

        contacts_table = contacts_table.reset_index(drop=True)

        return contacts_table

In [30]:
contacts = handle_contacts(df_dict['property_contacts'])

In [31]:
contacts

contacts  \
0   [{'company_id': 'c4b42de2-ab5c-5dcb-b4d0-aaf53...   
1                                                 NaN   
2                                                 NaN   
3                                                 NaN   
4   [{'company_id': '895e47ae-a9f3-5929-8b66-72e50...   
5                                                 NaN   
6                                                 NaN   
7   [{'has_address': True, 'has_email': True, 'has...   
8   [{'company_id': '2fd0640c-6871-5d51-81f6-3921a...   
9   [{'has_address': True, 'has_email': False, 'ha...   
10                                                NaN   
11                                                NaN   
12  [{'company_id': '88a90377-a1a8-5765-be19-48da0...   
13  [{'has_address': True, 'has_email': True, 'has...   
14  [{'company_id': '28d81ad8-54c9-5318-85bb-76ed5...   
15  [{'company_id': '681ff060-24fa-54d4-8149-f7bbd...   
16                                                NaN   
17  [{'company_id': 'fcc88e1d-13c4-5eb2-a58a-7392f...   
18                                                NaN   
19                                                NaN   
20                                                NaN   
21  [{'company_id': '89f1d598-3552-5785-b6db-182e8...   
22                                                NaN   
23                                                NaN   
24  [{'has_address': True, 'has_email': True, 'has...   
25  [{'has_address': True, 'has_email': True, 'has...   
26  [{'has_address': True, 'has_email': True, 'has...   
27                                                NaN   
28                                                NaN   
29  [{'company_id': 'cf3c7362-5e6c-56b0-b6b4-0a4f6...   
30                                                NaN   
31                                                NaN   
32  [{'has_address': True, 'has_email': False, 'ha...   
33                                                NaN   
34                                                NaN   
35  [{'company_id': '96e15980-e7f8-5f3f-b968-d92e3...   
36  [{'has_address': True, 'has_email': True, 'has...   
37                                                NaN   
38  [{'has_address': True, 'has_email': True, 'has...   
39                                                NaN   
40                                                NaN   
41                                                NaN   
42                                                NaN   
43  [{'has_address': True, 'has_email': False, 'ha...   
44  [{'has_address': True, 'has_email': True, 'has...   
45                                                NaN   
46  [{'has_address': True, 'has_email': True, 'has...   
47                                                NaN   
48                                                NaN   
49  [{'has_address': True, 'has_email': True, 'has...   

                                owner_id  is_individual_contact  is_contacts  \
0   c4b42de2-ab5c-5dcb-b4d0-aaf536879c77                  False         True   
1   b349b578-954f-5075-bc02-323eecfadd1e                   True        False   
2   6d230ebb-04e0-56e2-9dc1-a63194d71ac1                   True        False   
3   6fa11402-b879-5d1d-9793-8b1e3dd1ad3a                   True        False   
4   895e47ae-a9f3-5929-8b66-72e505a5e5c4                  False         True   
5   bed4ee32-c636-5afa-acac-64d7dcf89019                   True        False   
6   a7749aa2-76c3-579e-b1b6-f58ed77f8cbc                   True        False   
7   9d0d33aa-e1f7-501a-88f3-62f1bd28b5e5                  False         True   
8   2fd0640c-6871-5d51-81f6-3921a20f8703                  False         True   
9   13f0f21d-e6b5-57e8-b89b-0a40ce4de7c1                  False         True   
10  e2b61287-cecf-5f7b-bf37-7299dcacc3a3                   True        False   
11  c22341b0-5ed2-501e-b242-f83d5bbda9d6                   True        False   
12  88a90377-a1a8-5765-be19-48da0860ac36                  False         True   
13  2d72be1e-989f